# 说明
本次作业为二种类型，分别是选择题和实践题。其中选择题是不定项选择。实践题需要在提示的地方填上代码，跑通项目。

## 资料
做作业时可以参考以下资料。

PaddleSlim代码地址： [https://github.com/PaddlePaddle/PaddleSlim](https://github.com/PaddlePaddle/PaddleSlim)

文档地址：[https://paddlepaddle.github.io/PaddleSlim/](https://paddlepaddle.github.io/PaddleSlim/)

# 选择题
【1】定点化量化的优点有哪些？

A. 存带宽  B. 低功耗  C. 低计算资源  D. 低存储体积

【2】在常规蒸馏任务中，以下说法正确的是：

A. 只有teacher model的参数需要更新

B. 只有student model的参数需要更新

C. teacher model和student model 的参数都需要更新

D.teacher model和student model 的参数都不需要更新


【3】是否能用MobileNetv1蒸馏ResNet50？

A: 能

B: 不能

【4】下面方法哪些可以减少模型推理时间？

A. 只对权重weight进行量化

B. 对ResNet50模型进行蒸馏提高精度

C. 对模型进行裁剪，减少通道数

D. 对权重weight和激活进行量化，预测采用INT8计算


【5】NAS的三个关键要素是：

A. 搜索空间

B. 搜索算法

C. 模型优化

D. 模型评估



# 选择题答题卡

请将每道选择题的答案写在这里：

【1】ABCD

【2】B

【3】B

【4】BCD

【5】ABD





#  图像分类模型量化教程

该教程以图像分类模型MobileNetV1为例，说明如何快速使用[量化训练接口](https://paddlepaddle.github.io/PaddleSlim/api_cn/quantization_api.html#quant-aware)。
该示例包含以下步骤：

1. 导入依赖
2. 构建模型
3. 定义输入数据
4. 训练模型
5. 量化模型 ``这个步骤中需要添加代码``
6. 训练和测试量化后的模型

以下章节依次介绍每个步骤的内容。

## 0. 安装paddleslim



In [1]:
!pip install paddleslim

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 112kB 4.0MB/s ta 0:00:01


## 1. 导入依赖

PaddleSlim依赖Paddle1.7版本，请确认已正确安装Paddle，然后按以下方式导入Paddle和PaddleSlim:



In [2]:
import paddle
import paddle.fluid as fluid
import paddleslim as slim
import numpy as np


## 2. 构建模型

该章节构造一个用于对MNIST数据进行分类的分类模型，选用`MobileNetV1`，并将输入大小设置为`[1, 28, 28]`，输出类别数为10。
为了方便展示示例，我们在`paddleslim.models`下预定义了用于构建分类模型的方法，执行以下代码构建分类模型：



In [4]:
use_gpu = fluid.is_compiled_with_cuda()
exe, train_program, val_program, inputs, outputs = slim.models.image_classification("MobileNet", [1, 28, 28], 10, use_gpu=use_gpu)
place = fluid.CUDAPlace(0) if fluid.is_compiled_with_cuda() else fluid.CPUPlace()




## 3 定义输入数据

为了快速执行该示例，我们选取简单的MNIST数据，Paddle框架的`paddle.dataset.mnist`包定义了MNIST数据的下载和读取。
代码如下：



In [6]:

import paddle.dataset.mnist as reader
train_reader = paddle.batch(
        reader.train(), batch_size=128, drop_last=True)
test_reader = paddle.batch(
        reader.test(), batch_size=128, drop_last=True)
data_feeder = fluid.DataFeeder(inputs, place)




## 4. 训练和测试

先定义训练和测试函数，正常训练和量化训练时只需要调用函数即可。在训练函数中执行了一个epoch的训练，因为MNIST数据集数据较少，一个epoch就可将top1精度训练到95%以上。


In [8]:
def train(prog):
    iter = 0
    for data in train_reader():
        acc1, acc5, loss = exe.run(prog, feed=data_feeder.feed(data), fetch_list=outputs)
        if iter % 100 == 0:
            print('train iter={}, top1={}, top5={}, loss={}'.format(iter, acc1.mean(), acc5.mean(), loss.mean()))
        iter += 1
        
def test(prog):
    iter = 0
    res = [[], []]
    for data in test_reader():
        acc1, acc5, loss = exe.run(prog, feed=data_feeder.feed(data), fetch_list=outputs)
        if iter % 100 == 0:
            print('test iter={}, top1={}, top5={}, loss={}'.format(iter, acc1.mean(), acc5.mean(), loss.mean()))
        res[0].append(acc1.mean())
        res[1].append(acc5.mean())
        iter += 1
    print('final test result top1={}, top5={}'.format(np.array(res[0]).mean(), np.array(res[1]).mean()))



调用train函数训练分类网络，train_program是在第2步：构建网络中定义的

In [9]:
train(train_program)

train iter=0, top1=0.09375, top5=0.5625, loss=2.67902231216
train iter=100, top1=0.9140625, top5=1.0, loss=0.210934847593
train iter=200, top1=0.9140625, top5=1.0, loss=0.261303722858
train iter=300, top1=0.9609375, top5=0.984375, loss=0.188525319099
train iter=400, top1=0.9375, top5=1.0, loss=0.177733242512



调用test函数测试分类网络，val_program是在第2步：构建网络中定义的。

In [10]:
test(val_program)

test iter=0, top1=0.9921875, top5=1.0, loss=0.0384036004543
final test result top1=0.965144217014, top5=0.999499201775




## 5. 量化模型

按照配置在train_program和val_program中加入量化和反量化op.



In [13]:
TENSORRT_OP_TYPES = [
    'mul', 'conv2d', 'pool2d', 'depthwise_conv2d', 'elementwise_add',
    'leaky_relu'
]
TRANSFORM_PASS_OP_TYPES = ['conv2d', 'depthwise_conv2d', 'mul']

QUANT_DEQUANT_PASS_OP_TYPES = [
        "pool2d", "elementwise_add", "concat", "softmax", "argmax", "transpose",
        "equal", "gather", "greater_equal", "greater_than", "less_equal",
        "less_than", "mean", "not_equal", "reshape", "reshape2",
        "bilinear_interp", "nearest_interp", "trilinear_interp", "slice",
        "squeeze", "elementwise_sub", "relu", "relu6", "leaky_relu", "tanh", "swish"
    ]

_quant_config_default = {
    # weight quantize type, default is 'channel_wise_abs_max'
    'weight_quantize_type': 'channel_wise_abs_max',
    # activation quantize type, default is 'moving_average_abs_max'
    'activation_quantize_type': 'moving_average_abs_max',
    # weight quantize bit num, default is 8
    'weight_bits': 8,
    # activation quantize bit num, default is 8
    'activation_bits': 8,
    # ops of name_scope in not_quant_pattern list, will not be quantized
    'not_quant_pattern': ['skip_quant'],
    # ops of type in quantize_op_types, will be quantized
    'quantize_op_types': ['conv2d', 'depthwise_conv2d', 'mul'],
    # data type after quantization, such as 'uint8', 'int8', etc. default is 'int8'
    'dtype': 'int8',
    # window size for 'range_abs_max' quantization. defaulf is 10000
    'window_size': 10000,
    # The decay coefficient of moving average, default is 0.9
    'moving_rate': 0.9,
    # if True, 'quantize_op_types' will be TENSORRT_OP_TYPES
    'for_tensorrt': False,
    # if True, 'quantoze_op_types' will be TRANSFORM_PASS_OP_TYPES + QUANT_DEQUANT_PASS_OP_TYPES
    'is_full_quantize': False
}

In [14]:
place = exe.place
quant_program = slim.quant.quant_aware(train_program, place, config=_quant_config_default, scope=None, for_test=False)        #请在次数添加你的代码
val_quant_program = slim.quant.quant_aware(val_program, place, config=_quant_config_default, scope=None, for_test=False)    #请在次数添加你的代码

2020-04-06 13:05:32,600-INFO: quant_aware config {'weight_bits': 8, 'weight_quantize_type': 'channel_wise_abs_max', 'is_full_quantize': False, 'dtype': 'int8', 'moving_rate': 0.9, 'window_size': 10000, 'activation_bits': 8, 'quantize_op_types': ['conv2d', 'depthwise_conv2d', 'mul'], 'not_quant_pattern': ['skip_quant'], 'activation_quantize_type': 'moving_average_abs_max', 'for_tensorrt': False}
2020-04-06 13:05:33,643-INFO: quant_aware config {'weight_bits': 8, 'weight_quantize_type': 'channel_wise_abs_max', 'is_full_quantize': False, 'dtype': 'int8', 'moving_rate': 0.9, 'window_size': 10000, 'activation_bits': 8, 'quantize_op_types': ['conv2d', 'depthwise_conv2d', 'mul'], 'not_quant_pattern': ['skip_quant'], 'activation_quantize_type': 'moving_average_abs_max', 'for_tensorrt': False}


## 6 训练和测试量化后的模型¶
微调量化后的模型，训练一个epoch后测试。

In [15]:

train(quant_program)


train iter=0, top1=0.9453125, top5=0.9921875, loss=0.270684301853
train iter=100, top1=0.9609375, top5=1.0, loss=0.099116653204
train iter=200, top1=0.984375, top5=1.0, loss=0.0825728997588
train iter=300, top1=0.984375, top5=0.9921875, loss=0.0900387614965
train iter=400, top1=0.9453125, top5=1.0, loss=0.142843589187


测试量化后的模型，和3.2 训练和测试中得到的测试结果相比，精度相近，达到了无损量化。


In [16]:
test(val_quant_program)

test iter=0, top1=0.9453125, top5=1.0, loss=0.15757009387
final test result top1=0.975560903549, top5=0.999699532986
